In [1]:
from tensorflow.keras import optimizers, layers, losses
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
DB_PATH = r"dataset.csv"
TEST_SIZE = .2
VALIDATION_SIZE = .15
BATCH_SIZE = 32
EPOCHS = 100
DTYPE = 'float32'
LEARNING_RATE = 2e-3
RANDOM_STATE = 123
SCALE_COLS = ['trestbps', 'chol', 'thalach']

In [3]:
df = pd.read_csv(DB_PATH)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [4]:
X = df.drop('target', axis=1)
y = df['target']

In [5]:
X.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000


In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
dtypes: float64(1), int64(12)
memory usage: 30.9 KB


In [7]:
X.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
dtype: int64

In [8]:
scaler = MinMaxScaler()
X[SCALE_COLS] = scaler.fit_transform(X[SCALE_COLS])
X['age'] = X['age'] / 10

In [9]:
X = X.astype(DTYPE)
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,6.3,1.0,3.0,0.481132,0.244292,1.0,0.0,0.603053,0.0,2.3,0.0,0.0,1.0
1,3.7,1.0,2.0,0.339623,0.283105,0.0,1.0,0.885496,0.0,3.5,0.0,0.0,2.0
2,4.1,0.0,1.0,0.339623,0.178082,0.0,0.0,0.770992,0.0,1.4,2.0,0.0,2.0
3,5.6,1.0,1.0,0.245283,0.251142,0.0,1.0,0.816794,0.0,0.8,2.0,0.0,2.0
4,5.7,0.0,0.0,0.245283,0.520548,0.0,1.0,0.702290,1.0,0.6,2.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,5.7,0.0,0.0,0.433962,0.262557,0.0,1.0,0.396947,1.0,0.2,1.0,0.0,3.0
299,4.5,1.0,3.0,0.150943,0.315068,0.0,1.0,0.465649,0.0,1.2,1.0,0.0,3.0
300,6.8,1.0,0.0,0.471698,0.152968,1.0,1.0,0.534351,0.0,3.4,1.0,2.0,3.0
301,5.7,1.0,0.0,0.339623,0.011416,0.0,1.0,0.335878,1.0,1.2,1.0,1.0,3.0


In [10]:
X_train_, X_test, y_train_, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train_, y_train_, test_size=VALIDATION_SIZE, random_state=RANDOM_STATE)

In [11]:
class MLP(Model):
    def __init__(self, input_dim):
        super().__init__()
        self.activation = layers.LeakyReLU()
        self.dropout = layers.Dropout(.15)
        self.classifier = layers.Dense(1, activation='sigmoid')
        self.hidden_layers = [
            layers.Dense(input_dim, activation='linear'),
            layers.Dense(input_dim*2, activation='linear'),
            layers.Dense(input_dim*4, activation='linear'),
            layers.Dense(input_dim*8, activation='linear'),
            layers.Dense(input_dim*4, activation='linear'),
            layers.Dense(input_dim*2, activation='linear'),
            layers.Dense(input_dim, activation='linear')
        ]

    def call(self, x):
        for layer in self.hidden_layers:
            x = layer(x)
            x = self.activation(x)
            x = self.dropout(x)

        return self.classifier(x)

In [12]:
optimizer = optimizers.Adam(LEARNING_RATE)
loss = losses.BinaryCrossentropy()
mlp = MLP(input_dim=len(X.iloc[0]))
mlp.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
history = mlp.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val))


Epoch 1/100


7/7 [==============================] - 3s 53ms/step - loss: 0.7050 - accuracy: 0.5317 - val_loss: 0.6862 - val_accuracy: 0.5405
Epoch 2/100
7/7 [==============================] - 0s 9ms/step - loss: 0.6902 - accuracy: 0.5415 - val_loss: 0.6785 - val_accuracy: 0.6757
Epoch 3/100
7/7 [==============================] - 0s 9ms/step - loss: 0.6705 - accuracy: 0.6195 - val_loss: 0.6573 - val_accuracy: 0.7568
Epoch 4/100
7/7 [==============================] - 0s 9ms/step - loss: 0.6519 - accuracy: 0.6488 - val_loss: 0.6081 - val_accuracy: 0.7297
Epoch 5/100
7/7 [==============================] - 0s 8ms/step - loss: 0.6160 - accuracy: 0.6927 - val_loss: 0.5451 - val_accuracy: 0.7568
Epoch 6/100
7/7 [==============================] - 0s 9ms/step - loss: 0.5694 - accuracy: 0.7610 - val_loss: 0.4822 - val_accuracy: 0.8108
Epoch 7/100
7/7 [==============================] - 0s 8ms/step - loss: 0.4485 - accuracy: 0.8146 - val_loss: 0.4716 - val_accuracy: 0.7568
Epoch 8/100
7/7 [======

In [13]:
results = mlp.evaluate(X_test, y_test, batch_size=BATCH_SIZE)

2/2 [==============================] - 0s 6ms/step - loss: 0.6537 - accuracy: 0.8197


In [14]:
y_pred = np.where(mlp.predict(X_test)>.5, 1, 0)
print(classification_report(y_test, y_pred))

2/2 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

           0       0.85      0.77      0.81        30
           1       0.79      0.87      0.83        31

    accuracy                           0.82        61
   macro avg       0.82      0.82      0.82        61
weighted avg       0.82      0.82      0.82        61



In [15]:
gpc = GaussianProcessClassifier(kernel=RBF(), random_state=RANDOM_STATE)
gpc.fit(X_train_, y_train_)
gpc.score(X_test, y_test)

0.7213114754098361